In [132]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [2]:
class Dataset():
    def __init__(self,path):
        with open(path,"r") as file:
            self.text = file.read().split("@")[1:]
            
        self.data = {}
        
        for i in self.text:
            description = i.split()[0]
            value = i.split()[1:]
            
            if description == "attribute":
                description = value[0]
                value = value[1:]
            
            self.data[description] = value
            
            
        self.create_dataset()
        
    def create_dataset(self):
        
        values = []
        
        for i in self.data["data"]:
            value = i.split(",")
            
            value = [float(ii) for ii in value]
            
            values.append(value)
            
        numbers_i = [i for i in range(len(self.data["inputs"]))]
            
        self.dataset = pd.DataFrame(values,columns= numbers_i + ["out"])

        self.dataset["out"],_ = pd.factorize(self.dataset["out"])
        
        self.input = self.dataset.drop("out",axis = 1)
        self.out = self.dataset[["out"]]

In [22]:
class kd():
    def __init__(self,data):
        self.kd_tree = self.make_kd(data)
        
    def create_node(self):
        node = {"CORTE": None,
                "DIM": -1,
                "POINT": [],
                "MENOR": None,
                "MAIOR": None}

        return node
    
    def insert_kd(self,tree,point,n_dim):
        dim = tree["DIM"]

        if tree["CORTE"] == None:        
            l_tree = self.create_node()
            l_tree["DIM"] = (dim + 1) % n_dim
            r_tree = self.create_node()
            r_tree["DIM"] = (dim + 1) % n_dim

            p0 = tree["POINT"]
            tree["POINT"] = None

            if p0[dim] < point[dim]:
                l_tree["POINT"] = p0
                r_tree["POINT"] = point
            else:
                l_tree["POINT"] = point
                r_tree["POINT"] = p0


            median = np.median([p0[dim],point[[dim]]])[0]

            tree["CORTE"] = median
            tree["MENOR"] = l_tree
            tree["MAIOR"] = r_tree
        else:
            if point[dim] < tree["CORTE"]:
                self.insert_kd(tree["MENOR"],point,n_dim)
            else:
                self.insert_kd(tree["MAIOR"],point,n_dim)
    
    def make_kd(self,data):
        kd_tree = self.create_node()
        kd_tree["POINT"] = data.values[0]
        kd_tree["DIM"] = 0

        for point in data.values[1:]:
            self.insert_kd(kd_tree,point,n_dim=2)

        return kd_tree
    
    def get_deep(self):
        return get_deep_aux(self.kd_tree)
    
    def get_deep_aux(tree):
        if tree == None:
            return 0
        elif type(tree["POINT"]) == type(np.array([])):
            return 1
        else:
            return 1 + max(get_deep_aux(tree["MENOR"]),get_deep_aux(tree["MAIOR"]))
        
    def lines(self):
        plt.figure(figsize=(10,10))
        self.lines_aux(self.kd_tree)
        print("a")
        
    def lines_aux(self,tree,bounds = [[-100,100],[-100,100]]):
        corte = tree["CORTE"]
        dim = tree["DIM"]
        
        if type(tree["POINT"]) != type(None):
            1+1
        else:
            if dim == 0:
                a = bounds[1][0]
                b = bounds[1][1]
                
                plt.plot([corte,corte],[a,b], color = "r")
                lbound = [[bounds[0][0],corte], [bounds[1][0],bounds[1][1]]]
                rbound = [[corte,bounds[0][1]], [bounds[1][0],bounds[1][1]]]
                
            else:
                a = bounds[0][0]
                b = bounds[0][1]
                
                plt.plot([a,b],[corte,corte], color = "r")
                lbound = [[bounds[0][0],bounds[0][1]], [bounds[1][0],corte]]
                rbound = [[bounds[0][0],bounds[0][1]], [corte,bounds[1][1]]]
                
            if type(tree["MENOR"]) != type(None):
                self.lines_aux(tree=tree["MENOR"], bounds= lbound)
            
            if type(tree["MAIOR"]) != type(None):
                self.lines_aux(tree=tree["MAIOR"], bounds= rbound)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("out",1),
                                                    data["out"],test_size = 0.3, random_state= 1)
        

In [177]:
class x_NN():
    def __init__(self,data,test_size = 0.3):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data.drop("out",1),
                                                                                data["out"],test_size = test_size,
                                                                                random_state=1)
        
        self.kd_tree = kd(self.X_train)
        
    def knn_aux(self,tree,point,k_size, kneighbor, maior_distancia, check, dists):                    
        if tree == None:
            1+1
        elif type(tree["POINT"]) == type(np.array([])):
            dist = np.linalg.norm(point - tree["POINT"])
            
            if len(kneighbor) < k_size:
                kneighbor.append(tree["POINT"])
                dists.append(dist)
                
                if dist > maior_distancia[0]:
                    maior_distancia[0] = dist
                    maior_distancia[1] = len(kneighbor) - 1
            else:
                if dist < maior_distancia[0]:
                    tirar = maior_distancia[1]
                    
                    kneighbor.pop(tirar)
                    kneighbor.append(tree["POINT"])
                    
                    dists[tirar] = dist                    
                    maior_distancia[0] = np.max(dists)
                    maior_distancia[1] = np.argmax(dists)
            
            check[0] = check[0] + 1
                       
        else:
            corte = tree["CORTE"]
            dim = tree["DIM"]
            
            dif = abs(corte - point[dim])
            
            if len(kneighbor) < k_size or dif < maior_distancia[0]:
                menor = tree["MENOR"]
                self.knn_aux(tree = menor, point=point,k_size=k_size, 
                             kneighbor=kneighbor, maior_distancia = maior_distancia, check=check,dists=dists)
                maior = tree["MAIOR"]
                self.knn_aux(tree = maior, point=point,k_size=k_size, 
                             kneighbor=kneighbor, maior_distancia = maior_distancia, check=check,dists=dists)
            else:
                if point[dim] < corte:
                    menor = tree["MENOR"]
                    self.knn_aux(tree = menor, point=point,k_size=k_size, 
                                 kneighbor=kneighbor, maior_distancia = maior_distancia, check=check,dists=dists)
                else:
                    maior = tree["MAIOR"]
                    self.knn_aux(tree = maior, point=point,k_size=k_size, 
                                 kneighbor=kneighbor, maior_distancia = maior_distancia, check=check,dists=dists)
        
    def define_class(self,kneighbors):
        e = []
        for n in kneighbors:
            p = self.X_train
            for dim in range(len(n)):
                p = p[p[dim] == n[dim]]
            
            e.append(self.y_train[p.index[0]])
        
        contar = pd.DataFrame(e)[0].value_counts()
        contar = contar[contar == contar.iloc[0]]
        
        maiores = contar.index
        
        #return maiores[np.random.randint(len(maiores))]
        return maiores[0]
        
    def knn(self):
        start = time.time()
        
        objetivo = self.y_test.values
        classes = []
        
        print(0)        
        
        for i in self.X_test.values:
            kneighbor = []
            self.knn_aux(point = i, tree = self.kd_tree.kd_tree, k_size = 5, kneighbor= kneighbor, 
                         maior_distancia = [-1,-1], check = [0], dists = [])
            kneighbor = np.array(kneighbor)
            
            classes.append(self.define_class(kneighbor))
        
        print(1)
        classes = np.array(classes)
        
        acuracia = classes == objetivo
        acuracia = acuracia + 0
        acuracia = np.mean(acuracia)
        
        print(2)
        
        print("--- %s seconds ---" % (time.time() - start))
        
        return acuracia
        
x_NN(data[:100]).knn()

0
1
2
--- 0.19698524475097656 seconds ---


C:\Users\Franc\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


0.7333333333333333

In [5]:
data = Dataset(path = "dataset/banana.dat").dataset
data

,0,1,out
0,1.140,-0.114,0
1,-1.520,-1.150,1
2,-1.050,0.720,0
3,-0.916,0.397,1
4,-1.090,0.437,1
...,...,...,...
5295,0.335,1.390,1
5296,-1.700,-0.569,1
5297,2.640,1.140,1
5298,0.769,0.772,0
